# U-Net Model for RV Segmentation

## Training U-Net CNN

In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib as mplt
%matplotlib inline

from src import data,unet

Using TensorFlow backend.


In [2]:
from keras import utils

train_dir = "/home/ubuntu/training/TrainingSet"


images=[]
inner_masks=[]
outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    images += imgdata.labeled_images
    inner_masks += imgdata.endo_masks.values()
    outer_masks += imgdata.epi_masks.values()

images = np.asarray(images)[:,:,:,None].astype('float64')
i_masks = np.asarray(inner_masks)
o_masks = np.asarray(outer_masks)

dims = i_masks.shape
classes = len(set(i_masks[0].flatten()))
new_shape = dims + (classes,)
i_masks = utils.to_categorical(i_masks).reshape(new_shape)
o_masks = utils.to_categorical(o_masks).reshape(new_shape)

print("There are %d total training images." % len(images))
print("There are %d total inner masks." % len(inner_masks))
print("There are %d total outer masks." % len(outer_masks))


There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [4]:
height,width,_ = images[0].shape
dropout = 0.0

unet_conv = unet.UNet()

model = unet_conv.get_unet(height=height,width=width,channels=1,features=32,steps=3,padding='same')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 216, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 216, 256, 32)  320         input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 216, 256, 32)  0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 216, 256, 32)  9248        activation_1[0][0]               
___________________________________________________________________________________________

In [5]:
from keras.optimizers import Adam
from keras import backend as K

def dice_coef(y_true, y_pred):
        flat_y_true = K.flatten(y_true)
        flat_y_preds = K.flatten(y_pred)
        intersection = K.sum(flat_y_true*flat_y_preds)
        return (2. * intersection + 1.) / (K.sum(flat_y_true)+K.sum(flat_y_preds))


model.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy',metrics=[dice_coef])

In [6]:
#TODO: 1. split data into training and validation set
#      2. Augment the data
#      3. Train model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from math import ceil

seed = 0

validation_split=0.2

epochs = 100
batch_size = 20


split_index = int((1 - validation_split) * len(images))

train_steps = ceil(split_index / batch_size)
val_steps = ceil((len(images)-split_index )/batch_size)

train_images = images[:split_index]
train_inner_masks = i_masks[:split_index]

validation_images = images[split_index:]
validation_inner_masks = i_masks[split_index:]


data_gen_args = dict(rotation_range=180,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.01,
                     fill_mode='nearest')

train_images_datagen = ImageDataGenerator(**data_gen_args)
train_masks_datagen = ImageDataGenerator(**data_gen_args)


seed = 1
#height,width,channels = train_images[0].shape
#print(train_images[0].shape)
#print(train_inner_masks[0].shape)
train_images_datagen.fit(train_images,augment=True,seed=seed)
train_masks_datagen.fit(train_inner_masks,augment=True,seed=seed)

train_images_generator = train_images_datagen.flow(train_images, y=None, seed=seed)
train_masks_generator = train_images_datagen.flow(train_inner_masks, y=None, seed=seed)


train_generator = zip(train_images_generator, train_masks_generator)


/usr/local/lib/python3.5/dist-packages/Keras-2.0.8-py3.5.egg/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
/usr/local/lib/python3.5/dist-packages/Keras-2.0.8-py3.5.egg/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).


In [7]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.hdf5', verbose=1, save_best_only=True)

model.fit_generator(train_generator,steps_per_epoch=train_steps,epochs=epochs,verbose=1,callbacks=[checkpointer],
                   validation_data=(validation_images,validation_inner_masks),validation_steps=val_steps)

Epoch 1/100
10/10 [==============================] - 26s - loss: 0.7049 - dice_coef: 0.4945 - val_loss: 0.6939 - val_dice_coef: 0.4998
Epoch 2/100
10/10 [==============================] - 16s - loss: 0.6880 - dice_coef: 0.5028 - val_loss: 0.6815 - val_dice_coef: 0.5060
Epoch 3/100
10/10 [==============================] - 14s - loss: 0.6702 - dice_coef: 0.5120 - val_loss: 0.6578 - val_dice_coef: 0.5187
Epoch 4/100
10/10 [==============================] - 16s - loss: 0.6287 - dice_coef: 0.5356 - val_loss: 0.5947 - val_dice_coef: 0.5558
Epoch 5/100
10/10 [==============================] - 14s - loss: 0.5316 - dice_coef: 0.6007 - val_loss: 0.4654 - val_dice_coef: 0.6501
Epoch 6/100
10/10 [==============================] - 16s - loss: 0.4026 - dice_coef: 0.7237 - val_loss: 0.3515 - val_dice_coef: 0.7812
Epoch 7/100
10/10 [==============================] - 15s - loss: 0.3628 - dice_coef: 0.8195 - val_loss: 0.3330 - val_dice_coef: 0.8228
Epoch 8/100
10/10 [==============================] - 16

10/10 [==============================] - 16s - loss: 0.1627 - dice_coef: 0.9064 - val_loss: 0.1435 - val_dice_coef: 0.9229
Epoch 56/100
10/10 [==============================] - 15s - loss: 0.1773 - dice_coef: 0.8934 - val_loss: 0.1417 - val_dice_coef: 0.9144
Epoch 57/100
10/10 [==============================] - 16s - loss: 0.1605 - dice_coef: 0.9123 - val_loss: 0.1444 - val_dice_coef: 0.9075
Epoch 58/100
10/10 [==============================] - 16s - loss: 0.1637 - dice_coef: 0.9063 - val_loss: 0.1415 - val_dice_coef: 0.9149
Epoch 59/100
10/10 [==============================] - 15s - loss: 0.1607 - dice_coef: 0.9068 - val_loss: 0.1406 - val_dice_coef: 0.9191
Epoch 60/100
10/10 [==============================] - 16s - loss: 0.1563 - dice_coef: 0.9111 - val_loss: 0.1402 - val_dice_coef: 0.9189
Epoch 61/100
10/10 [==============================] - 15s - loss: 0.1552 - dice_coef: 0.9091 - val_loss: 0.1405 - val_dice_coef: 0.9251
Epoch 62/100
10/10 [==============================] - 16s - l

In [ ]:
model.load_weights('saved_models/weights.hdf5')



# Results of Endocardium Model

## Training
